In [1]:
import os
import re
import json
import anew
import math
import string
import nltk
import spacy
import datetime
import pandas as pd
from nltk.sentiment.util import mark_negation
from nltk.tokenize import WordPunctTokenizer, TweetTokenizer, sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
pd.set_option('display.max_colwidth', -1)
tok = WordPunctTokenizer()
tw_tok = TweetTokenizer()
stemmer = PorterStemmer()
# sent_tok = PunktSentenceTokenizer()
stop_words_eng = set(stopwords.words('english'))
nlp = spacy.load('en_core_web_sm')

In [3]:
ARO = 0      # (Constant) index for arousal vals
VAL = 1      # (Constant) index for valence vals

In [ ]:
# tweet data sample
"""
"id": 757000564941160448, 
"date": "2016-07-23", 
"time": "19:52:53", 
"timezone": "EDT", 
"user_id": 728381909991034884, 
"username": "UrsulaPflug", 
"tweet": ".@Kathleen_Wynne: Stop letting @Nestl\u00e9 suck Ontario dry for $3.71 per million litres of water  http://d.shpg.org/216672624t\u00a0", 
"replies": 0, 
"retweets": 0, 
"likes": 0, 
"hashtags": "", 
"link": "https://twitter.com/UrsulaPflug/status/757000564941160448", 
"retweet": null, 
"user_rt": "None", 
"mentions": "Kathleen_Wynne",
"location": ""
"""
# FDA data sample
"""
id_reference_num
delivery
date
text
"""

In [104]:
#input_doc_path = "./data/tweets_nestle_2016-07-23_2018-07-23.json"
input_doc_path = "./data/fda_warning_letter_analysis.csv"

with open(input_doc_path,"r") as f:
    if input_doc_path.endswith(".json"):
        doc = [json.loads(line) for line in f]
    elif input_doc_path.endswith(".csv"):
        doc = pd.read_csv(f, delimiter=',')

In [105]:
doc = pd.DataFrame(doc)

In [106]:
# rename column names to standardize data schema
doc = doc.rename(columns={"id_reference_num":"id","delivery":"source","username":"source","tweet":"text"})

In [107]:
# only select key columns and ignore rest of the dataset
doc = doc[["id","source","date","text"]]
doc.head()

id                                       source  \
0  fda_01        Electronic Mail                               
1  CMS # 559583  United Parcel Service\nSignature Required\n   
2  fda_02        UPS Overnight Delivery                        
3  fda_03        UPS OVERNIGHT DELIVERY                        
4  fda_04        EXPRESS DELIVERY                              

                 date  \
0  Oct. 11, 2018        
1  October 5, 2018      
2  July 24, 2018        
3  September 04, 2018   
4  SEPT 4, 2018         

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [125]:
# drop unnecessary columns
#doc.drop(["likes","replies","retweet","retweets","timezone","user_id","user_rt"],axis=1,inplace=True)

In [108]:
doc.dtypes

id        object
source    object
date      object
text      object
dtype: object

In [128]:
# drop duplicates
#test = test.drop_duplicates(["tweet"],keep="first")

In [109]:
exclude_punctuation = set([p for p in string.punctuation if p not in ["!",",",".","?",";"]])
# remove mentions, url, hashtag
def strip_text(t):
    #t = re.sub(r'\.?@\w+\:?',"",t)  
    t = re.sub(r'\d+','',t)
    t = re.sub('https?://.+',"",t)             # url
    t = re.sub('pic\.twitter\.com.+',"",t)     # url
    t = re.sub("[#]", " ", t)                  # remove hashtag
    t = t.lower()                              # lower case
    return ''.join(ch for ch in t if ch not in exclude_punctuation)   # remove unnessary punctuation

doc["text_clean"] = doc["text"].apply(lambda x: strip_text(x))
doc["text_clean"].head()

0    the center for tobacco products of the u.s. food and drug administration fda recently reviewed\n \nfda has determined that statements or representations made on the website \n \nadditionally, fda has determined that the products erimonabant hellocig eliquid and ecialis hellocig eliquid are unapproved new drugs in violation of section a of the fdc act  u.s.c. § a and misbranded under sections a, f, and f of the fdc act  u.s.c. §§ a, f, and f. \n \nsection tt violation\n \nfda has determined that statements or representations made on the website \n \n“making any express or implied statement or representation directed to consumers with respect to a tobacco product, in a label or labeling or through the media or advertising, that either conveys, or misleads or would mislead consumers into believing, that—\n   the product is approved by the food and drug administration;\n   the food and drug administration deems the product to be safe for use by consumers;\n   the product is endorsed b

In [130]:
#doc = doc.drop_duplicates(["tweet_clean"],keep="first")

In [44]:
#len(doc)

In [110]:
# word tokenizing
# doc["tweet_clean"] = doc["tweet_clean"].apply(lambda x: nlp(x.strip()))
# string.punctuation
def negation_process(d):
    sents = sent_tokenize(d)
    #sents = [tw_tok.tokenize(s) for s in sents]
    return [t for s in sents for t in mark_negation(document=tw_tok.tokenize(s),double_neg_flip=True,shallow=False) if len(t)>1]

#doc["tweet_clean"] = doc["tweet_clean"].apply(lambda x: [token for token in tok.tokenize(str(x)) if token.isalpha() and len(token)>1 and token not in stop_words_eng])
doc["text_clean"] = doc["text_clean"].apply(lambda x: negation_process(x))
doc["text_clean"].head()

0    [the, center, for, tobacco, products, of, the, food, and, drug, administration, fda, recently, reviewed, fda, has, determined, that, statements, or, representations, made, on, the, website, additionally, fda, has, determined, that, the, products, erimonabant, hellocig, eliquid, and, ecialis, hellocig, eliquid, are, unapproved, new, drugs, in, violation, of, section, of, the, fdc, act, and, misbranded, under, sections, and, of, the, fdc, act, and, section, tt, violation, fda, has, determined, that, statements, or, representations, made, on, the, website, making, any, express, or, implied, statement, or, representation, directed, to, consumers, with, respect, to, tobacco, product, in, label, or, labeling, or, through, the, media, or, ...]                                  
1    [cindie, kesler, ceopresident, farmersranchers, cooperative, association, of, ainsworth, main, street, ainsworth, ne, dear, ms, kesler, fda, conducted, an, inspection, on, may, and, of, your, nonlicensed, medi

In [111]:
# clean non-alphabic terms
def clean_non_aplha(terms):
    return list(set([x for x in terms if (x.endswith("_NEG") or x.isalpha()) and x not in stop_words_eng and len(x)>2]))

doc["text_clean"] = doc["text_clean"].apply(lambda x: clean_non_aplha(x))
#len(doc.loc[0,"text_clean"]) #1960 -> 1134

In [112]:
#doc.loc[0,"text_clean"]
len(doc["text_clean"][0])

449

In [ ]:
# document – a list of words/tokens, or a tuple (words, label).
# shallow – if True, the method will modify the original document in place.
# double_neg_flip – if True, double negation is considered affirmation 
# mark_negation(document=sent,double_neg_flip=True,shallow=False)

"""
def strip_tweet2(t):
    t = re.sub(r'@\w+',"",t)                   
    t = re.sub('https?://.+',"",t)             # url
    t = re.sub('pic\.twitter\.com.+',"",t)     # url
    t = re.sub("[#]", " ", t)            # remove hashtag
    t = t.lower()                              # lower case
    return t

sent = "i'm not happy. but she is fine "
sent2 = strip_tweet2(sent)
sents = sent_tokenize(sent2)
sent3 = [tw_tok.tokenize(s) for s in sents]
print(sent2)
print(sents)
print(sent3)
neg0 = mark_negation(document=sent3[0],double_neg_flip=True,shallow=False)
neg1 = mark_negation(document=sent3[1],double_neg_flip=True,shallow=False)
print(neg0 + neg1)
"""

In [113]:
def find_anew_term(x):
    term = -1
    # find term if it is negated
    if x[-4:] == "_NEG":
        x = x[:-4]
        term = anew.anew_neg_find_word(x)
        if term == -1:
            term = anew.anew_neg_find_stem(stemmer.stem(x))
    else:
        term = anew.anew_find_word(x)
        if term == -1:
            term = anew.anew_find_stem(stemmer.stem(x))
            if term == -1:
                term = anew.happiness_find_word(x)
                if term == -1:
                    term = anew.anew_ex_find_word(x)
                    if term == -1:
                        term = anew.anew_ex_find_stem(stemmer.stem(x))
    return term

def anew_term_process(x):
    anew_list = []
    for w in x:
        term = find_anew_term(w)
        if term != -1:
            anew_list.append(term)
    return anew_list
            
doc["anew_term"] = doc["text_clean"].apply(lambda x: anew_term_process(x))

In [114]:
doc[["text","text_clean","anew_term"]].head(5)

text  \
0  The Center for Tobacco Products of the U.S. Food and Drug Administration (FDA) recently reviewedhttp://www.hellocig.com and determined that the e-liquid products listed there are offered for sale or distribution to consumers in the United States. Under section 201(rr) of the Federal Food, Drug, and Cosmetic Act (FD&C Act) (21 U.S.C. § 321(rr)), as amended by the Family Smoking Prevention and Tobacco Control Act (Tobacco Control Act), these products are tobacco products because they are made or derived from tobacco and intended for human consumption. Certain tobacco products, including e-liquids, are subject to FDA jurisdiction under section 901(b) of the FD&C Act (21 U.S.C. § 387a(b)).\n \nFDA has determined that statements or representations made on the website http://www.hellocig.com regarding certain e-liquids violates section 301(tt) of the FD&C Act (21 U.S.C. 331(tt)) because they are directed to consumers and conveys, misleads, or would mislead consumers into believing that the tobacco products are approved by FDA. \n \nAdditionally, FDA has determined that the products E-Rimonabant HelloCig E-Liquid and E-Cialis HelloCig E-Liquid are unapproved new drugs in violation of section 505(a) of the FD&C Act (21 U.S.C. § 355(a)) and misbranded under sections 502(a), (f)(1), and (f)(2) of the FD&C Act (21 U.S.C. §§ 352(a), (f)(1), and (f)(2)). \n \nSection 301(tt) Violation\n \nFDA has determined that statements or representations made on the website http://www.hellocig.com regarding certain e-liquid products violates section 301(tt) of the FD&C Act (21 U.S.C. 331(tt)), which prohibits:\n \n“[m]aking any express or implied statement or representation directed to consumers with respect to a tobacco product, in a label or labeling or through the media or advertising, that either conveys, or misleads or would mislead consumers into believing, that—\n(1)   the product is approved by the Food and Drug Administration;\n(2)   the Food and Drug Administration deems the product to be safe for use by consumers;\n(3)   the product is endorsed by the Food and Drug Administration for use by consumers; or\n(4)   the product is safe or less harmful by virtue of—\n(A)  its regulation or inspection by the Food and Drug Administration; or\n(B)  its compliance with regulatory requirements set by the Food and Drug Administration…”\n \nSpecifically, certain e-liquid products are sold or distributed on the website to consumers in the United States with the statements: “99.9% pure nicotine is FDA approved product with FDA,” and “The 100% pure nicotine is FDA approved product with FDA”: DIY Pure Nicotine 5mL, DIY Pure Nicotine 10mL, DIY Pure Nicotine 30mL, DIY Pure Nicotine 60mL, DIY Pure Nicotine 250mL, and DIY Pure Nicotine 1L. Because these statements or representations on the website are directed to consumers and convey, mislead, or would mislead consumers into believing that the tobacco products are approved by FDA, they are in violation of section 301(tt) of the FD&C Act.\n \nMisbranded Tobacco Products\n \nFDA has also determined that certain e-liquid products are misbranded under section 903(a)(1) of the FD&C Act (21 U.S.C. § 387c(a)(1)) and/or section 903(a)(7)(A) of the FD&C Act (21 U.S.C. § 387c(a)(7)(A)). Section 903(a)(1) of the FD&C Act provides that a tobacco product is misbranded if its “labeling is false or misleading in any particular.” Section 903(a)(7) of the FD&C Act provides that a tobacco product distributed or offered for sale in any State is misbranded if its “advertising is false or misleading in any particular.” These e-liquid products are misbranded under section 903(a)(1) of the FD&C Act and/or section 903(a)(7)(A) of the FD&C Act as their labeling or advertising is false or misleading because it makes the statement or representation that the products are FDA approved.\n                                    \nUnapproved New Drug Violations\n \nThe E-Cialis HelloCig E-Liquid and E-Rimonabant HelloCig E-Liquid are “d

In [115]:
# calcuate average valance and arousal
def calc_val_aro_avg(terms):
    if terms is None or len(terms) == 0:
        return None
    fq_avg = 0             # term frequency average
    val_avg = 0            # valence average
    aro_avg = 0            # arousal average
    val_avg_ls = []
    aro_avg_ls = []
    val_std_avg = 0        # valence std average
    aro_std_avg = 0        # arousal std average
    val_std_p = []         # valence CDF (cummulative density function)
    aro_std_p = []         # arousal CDF 
    val_std_p_sum = 0
    aro_std_p_sum = 0
    num = len(terms)       # the number of terms
    res = {}
    for t in terms:
        if "fq" in t:
            fq_avg += float(t["fq"])
        if "avg" in t:
            val_avg += float(t["avg"][VAL])
            val_avg_ls.append(float(t["avg"][VAL]))
            aro_avg += float(t["avg"][ARO])
            aro_avg_ls.append(float(t["avg"][ARO]))
        if "std" in t:
            val_std_avg += float(t["std"][VAL])**2.0 + float(t["avg"][VAL])**2.0
            aro_std_avg += float(t["std"][ARO])**2.0 + float(t["avg"][ARO])**2.0
            val_std_p.append(1.0/math.sqrt(2.0*math.pi*float(t["std"][VAL])**2.0))
            aro_std_p.append(1.0/math.sqrt(2.0*math.pi*float(t["std"][ARO])**2.0))
    # calcuate averages
    fq_avg = fq_avg / num
    val_avg = val_avg / num
    aro_avg = aro_avg / num
    val_std_avg = math.sqrt((val_std_avg / num) - val_avg**2.0)
    aro_std_avg = math.sqrt((aro_std_avg / num) - aro_avg**2.0)
    val_std_p_sum = sum(val_std_p)
    aro_std_p_sum = sum(aro_std_p)
    # calculate final average for valence, arousal and frequence
    val_avg = 0
    aro_avg = 0
    for i in range(num):
        val_avg += (val_std_p[i]/val_std_p_sum)*val_avg_ls[i]
        aro_avg += (aro_std_p[i]/aro_std_p_sum)*aro_avg_ls[i]
    # calculate TOTAL average and std
    tot_avg = 0
    tot_std_avg = 0
    tot_avg = (aro_avg + val_avg)/2
    tot_std_avg = aro_std_avg**2.0 + aro_avg**2.0 + val_std_avg**2.0 + val_avg**2.0
    tot_std_avg = math.sqrt((tot_std_avg / 2) - tot_avg**2.0)
    tot_avg = 0
    tot_std_avg_aro_p = 1.0/math.sqrt(2.0*math.pi*aro_std_avg**2.0)
    tot_std_avg_val_p = 1.0/math.sqrt(2.0*math.pi*val_std_avg**2.0)
    tot_std_avg_sum = tot_std_avg_aro_p + tot_std_avg_val_p
    tot_avg = (tot_std_avg_aro_p/tot_std_avg_sum)*aro_avg + (tot_std_avg_val_p/tot_std_avg_sum)*val_avg
    # create response dictionary with attribute "avg","std","fq","rad"
    res["avg"] = [round(aro_avg,3),round(val_avg,3),round(tot_avg,3)]
    res["std"] = [round(aro_std_avg,3),round(val_std_avg,3),round(tot_std_avg,3)]
    if fq_avg >= 1:
        res["fq"] = round(fq_avg,2)
    else:
        res["fq"] = round(float(1),2)
    if fq_avg > 0:
        res["rad"] = 1.0 + math.log(fq_avg,2)
    else:
        res["rad"] = 1.0
    return res
                              
doc["val_aro_avg"] = doc["anew_term"].apply(lambda x: calc_val_aro_avg(x))

In [116]:
doc[["text","anew_term","val_aro_avg"]].head(5)

text  \
0  The Center for Tobacco Products of the U.S. Food and Drug Administration (FDA) recently reviewedhttp://www.hellocig.com and determined that the e-liquid products listed there are offered for sale or distribution to consumers in the United States. Under section 201(rr) of the Federal Food, Drug, and Cosmetic Act (FD&C Act) (21 U.S.C. § 321(rr)), as amended by the Family Smoking Prevention and Tobacco Control Act (Tobacco Control Act), these products are tobacco products because they are made or derived from tobacco and intended for human consumption. Certain tobacco products, including e-liquids, are subject to FDA jurisdiction under section 901(b) of the FD&C Act (21 U.S.C. § 387a(b)).\n \nFDA has determined that statements or representations made on the website http://www.hellocig.com regarding certain e-liquids violates section 301(tt) of the FD&C Act (21 U.S.C. 331(tt)) because they are directed to consumers and conveys, misleads, or would mislead consumers into believing that the tobacco products are approved by FDA. \n \nAdditionally, FDA has determined that the products E-Rimonabant HelloCig E-Liquid and E-Cialis HelloCig E-Liquid are unapproved new drugs in violation of section 505(a) of the FD&C Act (21 U.S.C. § 355(a)) and misbranded under sections 502(a), (f)(1), and (f)(2) of the FD&C Act (21 U.S.C. §§ 352(a), (f)(1), and (f)(2)). \n \nSection 301(tt) Violation\n \nFDA has determined that statements or representations made on the website http://www.hellocig.com regarding certain e-liquid products violates section 301(tt) of the FD&C Act (21 U.S.C. 331(tt)), which prohibits:\n \n“[m]aking any express or implied statement or representation directed to consumers with respect to a tobacco product, in a label or labeling or through the media or advertising, that either conveys, or misleads or would mislead consumers into believing, that—\n(1)   the product is approved by the Food and Drug Administration;\n(2)   the Food and Drug Administration deems the product to be safe for use by consumers;\n(3)   the product is endorsed by the Food and Drug Administration for use by consumers; or\n(4)   the product is safe or less harmful by virtue of—\n(A)  its regulation or inspection by the Food and Drug Administration; or\n(B)  its compliance with regulatory requirements set by the Food and Drug Administration…”\n \nSpecifically, certain e-liquid products are sold or distributed on the website to consumers in the United States with the statements: “99.9% pure nicotine is FDA approved product with FDA,” and “The 100% pure nicotine is FDA approved product with FDA”: DIY Pure Nicotine 5mL, DIY Pure Nicotine 10mL, DIY Pure Nicotine 30mL, DIY Pure Nicotine 60mL, DIY Pure Nicotine 250mL, and DIY Pure Nicotine 1L. Because these statements or representations on the website are directed to consumers and convey, mislead, or would mislead consumers into believing that the tobacco products are approved by FDA, they are in violation of section 301(tt) of the FD&C Act.\n \nMisbranded Tobacco Products\n \nFDA has also determined that certain e-liquid products are misbranded under section 903(a)(1) of the FD&C Act (21 U.S.C. § 387c(a)(1)) and/or section 903(a)(7)(A) of the FD&C Act (21 U.S.C. § 387c(a)(7)(A)). Section 903(a)(1) of the FD&C Act provides that a tobacco product is misbranded if its “labeling is false or misleading in any particular.” Section 903(a)(7) of the FD&C Act provides that a tobacco product distributed or offered for sale in any State is misbranded if its “advertising is false or misleading in any particular.” These e-liquid products are misbranded under section 903(a)(1) of the FD&C Act and/or section 903(a)(7)(A) of the FD&C Act as their labeling or advertising is false or misleading because it makes the statement or representation that the products are FDA approved.\n                                    \nUnapproved New Drug Violations\n \nThe E-Cialis HelloCig E-Liquid and E-Rimonabant HelloCig E-Liquid are “d

In [117]:
# generate hover over tips
def create_tips(row):
    tweet = row["text"]
    for term in row["anew_term"]:
        if "dict" in term:
            #if term["dict"] == "anew-neg":
            if term["avg"][VAL] >= 5:
                #tweet = re.sub(r'\s+'+term["word"]+'\s+'," [N]"+term["word"]+"[E] ",tweet)
                tweet = re.sub(r'\s+'+term["word"]+'\s+'," [PV]"+term["word"]+"[E] ",tweet)
            else:
                #tweet = re.sub(r'\s+'+term["word"]+'\s+'," [A]"+term["word"]+"[E] ",tweet)
                tweet = re.sub(r'\s+'+term["word"]+'\s+'," [NV]"+term["word"]+"[E] ",tweet)
    return tweet

doc["tip"] = doc.apply(create_tips,axis=1)

In [118]:
doc[["text_clean","anew_term","val_aro_avg","tip"]].head(5)

text_clean  \
0  [claims, similar, approval_NEG, for_NEG, convey, uses, effective_NEG, provision, licensed_NEG, presence, measures, light, civil_NEG, cure, representations_NEG, declare, making, deems, hellocig, confirmed, caa, exempt, unapproved, sildenafil, placed, container, compliance, suffer, partially, active, category, actions, direction, ele, corrective, noted, sale, bottle, seizure_NEG, warnings, recently, regulatory, misleading, levels, ingredients, recognized_NEG, nicotine, including, any_NEG, discussed, filed_NEG, applications_NEG, requirements, following, interact, well, number, laboratory, prescription, constitute_NEG, implementing, commerce, days, subject, users, sections, protection, tablets_NEG, labeling, establishments, extent, these_NEG, find, consequently, amazing, either, knowing, office, describing, erimonobant, homepage, made_NEG, similarly, described, room, less, regulation, disease, one, administration, suggests, determined, andor_NEG, our_NEG, reasons, maintaining, use_NEG, content, are_NEG, lower, ...]                  
1  [similar, if_NEG, on_NEG, for_NEG, batch, presence, adequately, aid, can_NEG, conducting, costs, federal, provide, allinclusive_NEG, sampling, planned, records_NEG, manufactured, caused, assess, compliance, explanation, discussed_NEG, causes, actions, limits_NEG, plans, reason, corrective, noted, prepare, report, reinspectionrelated, packing, approved_NEG, sequencing, must, chlortetracycline, dear, including, identification, any_NEG, documented, contaminated, fifteen, deviations, requirements, following, machine, updated, reinspection, sources, laboratory, unless, discusses, implementing, days, subsequently, administrative, improper, subject, sections, resulted, adequately_NEG, responsible, sandhills, designee, materially, fees, occurred, hazardsfor, assessing, writing, code, suite, health_NEG, find, operator, death_NEG, overall, applies, office, results, added, implementation, located, assuring, corrections, horse, collect, ensuring, situations, unmarked, one, administration, determined, mistakenly, sops, cover, street, ...]   
2  [monensin_NEG, similar, if_NEG, form_NEG, reported, on_NEG, approval_NEG, for_NEG, federal_NEG, inside, cardiac, packed, appear_NEG, batch, presence, schedule_NEG, horses, can_NEG, conducting, costs, federal, provide, allinclusive_NEG, placed, planned, manufactured, public, assess, compliance, explanation, humans, causes, cleaning_NEG, mill, plans, reason, noted, muscles, prepare, exit, report, box, reinspectionrelated, packing, approved_NEG, must, dear, including, firm_NEG, identification, batching, any_NEG, ataxia, requires, patterns, maintenance_NEG, requirements, following, feeds_NEG, inspectional_NEG, production, well, nonmedicated_NEG, number, reinspection, laboratory, reliable, unless, livestock, implementing, notes, days, administrative, subject, steer, sections, responsible, labeling, designee, materially, consequence, fees, occurred, documenting_NEG, assessing, writing, code, suite, health_NEG, amount_NEG, find, adulteration, death_NEG, overall, applies, results, implementation, adam, other_NEG, located, ...]          
3  [reported, off_NEG, unlike, proteins, for_NEG, uses, expected, important, salmonella, heattreated, presence, elderly, measures, conditions, that_NEG, confirmed, federal, provide, allinclusive_NEG, ate, placed, september, manufactured, immune, authorizes_NEG, public, i_NEG, compliance, explanation, suffer, otherwise, humans, active, minimal, causes, cleaning_NEG, actions, b_NEG, spleen, place, never, reason, scientific, corrective, copies, correction, shortterm, prepare, regulatory, packing, sequencing, must, whole, including, any_NEG, contaminated, discussed, fifteen, requirements, following, isolated, expressed_NEG, well, number, laboratory, road, unless, days, commerce, class, areasonable, february, lots, beef, subject, sections, responsible, atlu, maddox, designee, unable, these_NEG, hazardsfor, complaint, readytoeat, writin

In [119]:
def separate_columns(col_name,src_col):
    if src_col is not None and col_name in src_col:
        return src_col[col_name]
    else:
        return None

doc["avg"] = doc["val_aro_avg"].apply(lambda x: separate_columns("avg",x))
doc["std"] = doc["val_aro_avg"].apply(lambda x: separate_columns("std",x))
doc["fq"] = doc["val_aro_avg"].apply(lambda x: separate_columns("fq",x))
doc["rad"] = doc["val_aro_avg"].apply(lambda x: separate_columns("rad",x))

In [96]:
doc.dtypes

id             object 
source         object 
date           object 
text           object 
text_clean     object 
anew_term      object 
val_aro_avg    object 
tip            object 
avg            object 
std            object 
fq             float64
rad            float64
dtype: object

In [120]:
# rename columns 
#doc = doc.rename(columns={"location":"geo","text":"raw","username":"name","hashtags":"tags","tweet_clean":"term","anew_term":"anew"})
doc = doc.rename(columns={"text":"raw","source":"name","text_clean":"term","anew_term":"anew"})

In [98]:
doc.dtypes

id             object 
name           object 
date           object 
raw            object 
term           object 
anew           object 
val_aro_avg    object 
tip            object 
avg            object 
std            object 
fq             float64
rad            float64
dtype: object

In [121]:
# add additional columns to meet required columns
doc["dup"] = 0
doc["geo"] = ""
doc["fmt"] = doc["tip"]
doc["body"] = doc["term"].apply(lambda x:" ".join(x))

In [144]:
# format datetime and create time column
#doc["date"] = doc["date"] + " " + doc["time"]

In [145]:
"""
def format_time(x):
    try:
        return datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").strftime("%a %B %d %H:%M:%S %Y")
    except:
        return ""

doc["time"] = doc["date"].apply(lambda x: format_time(x))
"""

In [78]:
#doc[["date","time"]].head()

In [122]:
doc.drop(["val_aro_avg","date"],axis=1,inplace=True)
doc.dtypes

id      object 
name    object 
raw     object 
term    object 
anew    object 
tip     object 
avg     object 
std     object 
fq      float64
rad     float64
dup     int64  
geo     object 
fmt     object 
body    object 
dtype: object

In [123]:
# escape double quotes
def escape_double_quotes(x):
    x = x.replace('"',"'")
    #x = x.replace('\/',"/")
    return x

doc["raw"] = doc["raw"].apply(lambda x: escape_double_quotes(x))
doc["tip"] = doc["tip"].apply(lambda x: escape_double_quotes(x))
doc["fmt"] = doc["fmt"].apply(lambda x: escape_double_quotes(x))

In [124]:
doc["tip"]

0     The Center for Tobacco Products of the U.S. Food and Drug Administration (FDA) recently reviewedhttp://www.hellocig.com and [PV]determined[E] that the e-liquid products [PV]listed[E] there are [PV]offered[E] for [PV]sale[E] or distribution to consumers in the United States. Under [PV]section[E] 201(rr) of the Federal Food, Drug, and Cosmetic Act (FD&C Act) (21 U.S.C. § 321(rr)), as amended by the Family Smoking Prevention and Tobacco Control Act (Tobacco Control Act), these products are [NV]tobacco[E] products because they are [PV]made[E] or derived from [NV]tobacco[E] and intended for human consumption. Certain [NV]tobacco[E] products, including e-liquids, are [PV]subject[E] to FDA jurisdiction under [PV]section[E] 901(b) of the FD&C Act (21 U.S.C. § 387a(b)).\n \nFDA has [PV]determined[E] that statements or representations [PV]made[E] on the [PV]website[E] http://www.hellocig.com regarding [PV]certain[E] e-liquids violates [PV]section[E] 301(tt) of the FD&C Act (21 U.S.C. 331(t

In [125]:
test_json = doc.to_json(orient='records')
test_json

'[{"id":"fda_01","name":"Electronic Mail","raw":"The Center for Tobacco Products of the U.S. Food and Drug Administration (FDA) recently reviewedhttp:\\/\\/www.hellocig.com and determined that the e-liquid products listed there are offered for sale or distribution to consumers in the United States. Under section 201(rr) of the Federal Food, Drug, and Cosmetic Act (FD&C Act) (21 U.S.C. \\u00a7 321(rr)), as amended by the Family Smoking Prevention and Tobacco Control Act (Tobacco Control Act), these products are tobacco products because they are made or derived from tobacco and intended for human consumption. Certain tobacco products, including e-liquids, are subject to FDA jurisdiction under section 901(b) of the FD&C Act (21 U.S.C. \\u00a7 387a(b)).\\n \\nFDA has determined that statements or representations made on the website http:\\/\\/www.hellocig.com regarding certain e-liquids violates section 301(tt) of the FD&C Act (21 U.S.C. 331(tt)) because they are directed to consumers and 